# 기본 구조를 활용한 제안 모델 작성해보기

### 문제
- 이미지를 입력으로 받아 클래스를 분류하는 모델을 Pytorch로 구현해보세요.
- 이미지 데이터는 500x500으로 변환해야해요.
- 손실 함수는 교차엔트로피 손실, 최적화 함수는 Adam을 사용해야 합니다.
- 출력 레이어는 Fully Connected

### **숙제 정보**

■ 난이도 : 🔴상

■ 실습 범위 : 2주차

■ 사용 언어 및 라이브러리 : pytorch

In [7]:
#라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim #최적화 함수들을 포함하고 있으며, 이 코드에서는 Adam을 불러옴
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset

In [20]:
#데이터 전처리 설정
# 이미지를 500x500으로 변환해야 하므로, torchvision.transforms를 사용해 이미지를 500x500으로 리사이즈하고, 텐서로 변환
transform = transforms.Compose([
    transforms.Resize((500,500)), #이미지를 500x500으로 크기 조절
    transforms.ToTensor() #이미지를 PyTorch 텐서로 변환, 이 과정에서 픽셀 값이 0과 1 사이로 정규화
])

In [9]:
train_dataset = datasets.FakeData(size = 1000, transform = transform, num_classes = 3)
test_dataset = datasets.FakeData(size=200, transform=transform, num_classes=3)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32,shuffle=False)

#datasets.FakeData: 임의로 생성된 데이터셋을 만듦. size는 데이터셋 크기, num_classes는 분류할 클래스 수

In [17]:
#CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 125 * 125, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 125 * 125)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
        
#모델 초기화
model = SimpleCNN()


In [18]:
#손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
#모델 훈련

#훈련 루프
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print("학습 종료")

Epoch 1, Loss: 5.042930297553539
Epoch 2, Loss: 1.0964488349854946
Epoch 3, Loss: 1.0711977370083332
Epoch 4, Loss: 0.7225117264315486
Epoch 5, Loss: 0.19596216385252774
Epoch 6, Loss: 0.03326049716270063
Epoch 7, Loss: 0.007632721033587586
Epoch 8, Loss: 0.0011785315709857969
Epoch 9, Loss: 0.0004904147472188924
Epoch 10, Loss: 0.00020744048015330918
학습 종료
